In [9]:
#Import required Google Earth Engine python packages and check if they work in python environment
import ee

In [12]:
#Import required Google Earth Engine python packages and check if they work in python environment
import geemap
Map = geemap.Map()

#Import Tenneessee River Boundary
TN_River = ee.FeatureCollection("users/pjf927/TN_River_Sample")

#Import Sentinel Satellite Imagery
Sentinel_2 = ee.ImageCollection('COPERNICUS/S2_SR')

#Filter out only pixles in the Tennessee River, an image on a particular date, a scene without cloud cover
image = ee.Image(Sentinel_2 \
                  .filterBounds(TN_River) \
                  .filterDate('2020-04-10', '2020-04-20') \
                  .sort('CLOUD_COVER') \
                  .first() \
                  .clip(TN_River))

#Set the center of the map to the location of the Tennessee River
Map.setCenter(-85.0323, 35.3587, 10)

#Add the Tennessee River outline to the map
Map.addLayer(TN_River)

#Visualize only the red, green, and blue bands
visParams = {
  'bands':['B4','B3','B2'],
  'min': 315.209590909109,
  'max': 996.6863901783613
}

#Display the river image in the Map
Map.addLayer(image, visParams, 'clipped composite')

#Create a variable for each band
blue = image.select('B2')
green = image.select('B3')
red = image.select('B4')
nir = image.select('B8')

#Compute the Normalized Difference Suspended Sediment Index (NDSSI)
ndssi = blue.subtract(nir).divide(blue.add(nir)).rename('NDSSI')

#Compute the Normalized Difference Turbidity Index (NDTI)
ndti = red.subtract(blue).divide(red.add(blue)).rename('NDTI')

#Compute the Normalized Suspended Material Index (NSMI)
nsmi = red.add(green.subtract(blue)).divide(red.add(green.add(blue))).rename('NSMI')

#Display the result
ndssiParams = {'min': -0.6655212627175484, 'max': 0.36753527747273956, 'palette': ['#BF4000', '#AB552F', '#8F7D63' ,'#73A597', '#57CDCB', '#3BF5FF']}
Map.addLayer(ndssi, ndssiParams, 'NDSSI')
ndtiParams = {'min': -0.048428901865121526, 'max': 0.1437408950820568, 'palette': ['#3BF5FF', '#57CDCB', '#73A597', '#8F7D63', '#AB552F', '#BF4000']}
Map.addLayer(ndti, ndtiParams, 'NDTI')
nsmiParams = {'min': -0.37542954986424937, 'max': 0.5203360124786097, 'palette': ['#3BF5FF', '#57CDCB', '#73A597', '#8F7D63', '#AB552F', '#BF4000']}
Map.addLayer(nsmi, nsmiParams, 'NSMI')
Map

Map(center=[35.3587, -85.0323], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…